In [1]:
from IPython.display import display, HTML
display(HTML("<style>div.container { width:100% !important; }</style>"))

# 인공신경망 딥러닝 모델 구현

In [2]:
import tensorflow
tensorflow.__version__

'2.13.0'

## iris 데이터 불러오기

In [3]:
from sklearn import datasets

In [4]:
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target

In [5]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y, test_size=0.3,
                                                    random_state=1)

## 싸이킷런을 이용한 구현

In [6]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,30), activation="logistic", solver="adam",
                    max_iter=1000) 

In [7]:
mlp.fit(train_X, train_y)

MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 30), max_iter=1000)

In [8]:
mlp.score(test_X, test_y)

1.0

In [9]:
pred = mlp.predict(test_X)

In [10]:
import pandas as pd
pd.crosstab(pred, test_y) #교차분류표
# pd.crosstab(np.argmax(pred, axis=1), np.argmax(test_y, axis=1)) #교차분류표

col_0,0,1,2
row_0,,,
0,14,0,0
1,0,18,0
2,0,0,13


## 텐서플로우를 이용한 구현

In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
x = tf.placeholder(tf.float32, [None, 4])
y = tf.placeholder(tf.float32, [None, 3])

In [13]:
W1 = tf.Variable(tf.random_normal([4, 50]))
b1 = tf.Variable(tf.zeros([50]))
h1 = tf.nn.sigmoid(tf.matmul(x, W1)+b1)

In [14]:
W2 = tf.Variable(tf.random_normal([50, 30]))
b2 = tf.Variable(tf.zeros([30]))
h2 = tf.nn.sigmoid(tf.matmul(h1, W2)+b2)

In [15]:
W3 = tf.Variable(tf.random_normal([30, 3]))
b3 = tf.Variable(tf.zeros([3]))
h3 = tf.nn.softmax(tf.matmul(h2, W3)+b3)

In [16]:
cross_entropy = -tf.reduce_sum(y*tf.log(h3), reduction_indices=[1])
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer().minimize(loss)

In [17]:
import pandas as pd

feed_train = {x: train_X, y: pd.get_dummies(train_y).values}

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(1000):
    sess.run(train, feed_dict=feed_train)

    if i%100 == 0:
        tr_loss = sess.run(loss, feed_dict=feed_train)
        print("Step %d, %.5f" % (i, tr_loss))

Step 0, 3.29460
Step 100, 0.50687
Step 200, 0.26487
Step 300, 0.17835
Step 400, 0.12432
Step 500, 0.10359
Step 600, 0.09261
Step 700, 0.08554
Step 800, 0.08034
Step 900, 0.07632


In [18]:
correct_prediction = tf.equal(tf.argmax(h3, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
feed_test = {x: test_X, y: pd.get_dummies(test_y).values}
print(sess.run(accuracy, feed_dict=feed_test))

1.0


In [19]:
sess.close()

## 케라스를 이용한 구현

In [20]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [21]:
from tensorflow.keras.layers import InputLayer, Dense
model.add(InputLayer(4))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))

In [22]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [23]:
model.fit(train_X, train_y, epochs=300, verbose=0)

In [24]:
model.evaluate(test_X, test_y) # 출력값은 loss와 accuracy

C:\Users\kosa\anaconda3\lib\site-packages\keras\src\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[0.062196592489878334, 1.0]

## 파이토치를 이용한 구현

In [25]:
# # CPU 버전 설치 (최신 버전)
# ! pip install torch

# # GPU 지원 버전 설치 (최신 버전)
# ! pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html

In [26]:
import torch
print(torch.__version__)

2.0.1+cpu


In [27]:
from sklearn import datasets
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y, test_size=0.3,
                                                    random_state=1)

In [28]:
import torch
import torch.nn as nn

# 모델 클래스 정의
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 3)
        self.softmax = nn.Softmax(dim=1)
        # 가중치 초기화 (Xavier 초기화)
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

# 모델 생성
model = CustomModel()

In [29]:
# 모델 정의를 아래처럼 Sequential을 이용할 수 있음
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(4, 50), nn.Sigmoid(),
    nn.Linear(50, 30), nn.Sigmoid(),
    nn.Linear(30, 3), nn.Softmax(dim=1)
)

In [30]:
# 손실 함수 및 옵티마이저 정의
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# 모델 학습
for epoch in range(1000):
    inputs = torch.from_numpy(train_X).float()
    labels = torch.from_numpy(train_y).long()
    
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

In [31]:
# 모델 평가
with torch.no_grad():
    inputs = torch.from_numpy(test_X).float()
    labels = torch.from_numpy(test_y).long()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == labels).sum().item() / len(test_y)

print(f"Test Loss:{loss.item()}, Test Accuracy:{accuracy}")

Test Loss:0.5719500780105591, Test Accuracy:0.9777777777777777


### 파이토치의 Sequential 이용

In [32]:
from sklearn import datasets
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y, test_size=0.3,
                                                    random_state=1)

In [33]:
# 넘파이 배열을 토치 텐서로 변환
import torch

train_X = torch.tensor(train_X, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.int64)
test_X = torch.tensor(test_X, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.int64)

In [34]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# 모델 정의
model = nn.Sequential(
    nn.Linear(4, 50), nn.Sigmoid(),
    nn.Linear(50, 30), nn.Sigmoid(),
    nn.Linear(30, 3), nn.Softmax(dim=1)
)

# 가중치 초깃값 지정
for layer in model:
    if isinstance(layer, nn.Linear):
        nn.init.xavier_uniform_(layer.weight)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [35]:
# 모델 학습
for epoch in range(300):
    outputs = model(train_X)
    loss = criterion(outputs, train_y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# 모델 평가
with torch.no_grad():
    test_outputs = model(test_X)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == test_y).sum().item() / test_y.size(0)
    print("Loss: {}, Accuracy: {}".format(loss.item(), accuracy))

Loss: 0.6756656169891357, Accuracy: 0.9777777777777777
